In [7]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import time

# Load and Prepare Historical Data
file_path_1 = 'transactions_1.csv'
file_path_2 = 'transactions_2.csv'
transactions = load_and_prepare_data(file_path_1, file_path_2)

def load_and_prepare_data(file_path_1, file_path_2):
    transactions_1 = pd.read_csv(file_path_1)
    transactions_2 = pd.read_csv(file_path_2)

    transactions_1 = transactions_1.rename(columns={'f0_': 'count'})

    transactions = pd.concat([transactions_1, transactions_2])
    
    return transactions

# Calculate thresholds
def calculate_thresholds(transactions, num_std_dev=3):
    status_stats = transactions.groupby('status')['count'].agg(['mean', 'std']).reset_index()
    status_stats['threshold'] = status_stats['mean'] + num_std_dev * status_stats['std']
    return status_stats

status_stats = calculate_thresholds(transactions)

# Detect anomalies
def detect_anomalies(new_data, status_stats):
    anomalies = {}
    for _, row in status_stats.iterrows():
        status = row['status']
        threshold = row['threshold']
        total_count = new_data[new_data['status'] == status]['count'].sum()
        if total_count > threshold:
            anomalies[status] = total_count
    return anomalies

# Send email alerts
def send_alert(anomalies, recipient_email):
    sender_email = "alert@cloudwalk.com"
    sender_password = "password" 
    
    subject = "Alert: Transaction Anomalies Detected"
    message = "The following anomalies were detected:\n\n"
    for status, count in anomalies.items():
       message += f"{status.capitalize()} transactions: {count}\n"
    
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'plain'))
    
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587) 
        server.starttls() 
        server.login(sender_email, sender_password) 
        server.sendmail(sender_email, recipient_email, msg.as_string())
        server.close() 
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Process new data real time
def process_new_data(new_data):
    global transactions
    transactions = pd.concat([transactions, new_data])
    anomalies = detect_anomalies(new_data, status_stats)
    if anomalies:
        recipient_email = "daniel.vargas@cloudwalk.com"
        send_email_alert(anomalies, recipient_email)